In [15]:
# inspect D:\Life\Academic\Skripsi\Code\data\wikipediasmallLM\data\train-00000-of-00145-437d77a320cee9e2.parquet

import pandas as pd

df = pd.read_parquet("D:\\Life\\Academic\\Skripsi\\Code\\data\\wikipediasmallLM\\data\\train-00000-of-00145-437d77a320cee9e2.parquet")
print(df.head())

   id           title                                               text  \
0   0  Deaths in 2022  The following notable deaths occurred in 2022....   
1   1         YouTube  YouTube is a global online video sharing and s...   
2   2         YouTube  In October 2006, YouTube was bought by Google ...   
3   3         YouTube  Since its purchase by Google, YouTube has expa...   
4   4         YouTube  YouTube has had an unprecedented social impact...   

                                            url   wiki_id        views  \
0  https://en.wikipedia.org/wiki?curid=69407798  69407798  5674.449219   
1   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   
2   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   
3   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   
4   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   

   paragraph_id  langs                                                emb  
0             0     38

In [13]:
# convert all emb to list
df["emb"] = df["emb"].apply(lambda x: x.tolist())

In [ ]:
# confirm id is unique
df['id'].is_unique

True

In [ ]:
# count unique id
len(df['id'].unique())

242538

In [ ]:
# initialize qdrant client
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")
COLLECTION_NAME = "wikipedia"

CountResult(count=1455228)

In [ ]:
# count number of vectors in collection
client.count(COLLECTION_NAME, exact=True)

In [ ]:
# get collection info
info = client.get_collection(COLLECTION_NAME)
print(info)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=1455228 points_count=1455228 segments_count=5 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=8, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=True, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=10000, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=4), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_confi

In [ ]:
# delete collection
client.delete_collection(COLLECTION_NAME)

True

In [ ]:
# patch collection to change HNSW m value to 8
client.update_collection(
    collection_name=COLLECTION_NAME,
    hnsw_config=models.HnswConfigDiff(
        m=8,
    ),
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=10000,
    )
)

In [ ]:
# force optimizer to run again
client.update_collection(
    collection_name="wikipedia",
    optimizer_config=models.OptimizersConfigDiff(),
)